In [1]:
#imports
import pandas as pd
import numpy as np
from collections import Counter
import re
import csv
import seaborn as sns
import matplotlib.pylab as plt
from scipy.stats import pearsonr

import warnings
warnings.filterwarnings('ignore')
import os

In [2]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='camligorkem', api_key='ls8JEHoNuI6dj5AHrzy2')

In [3]:
topic_distribution = pd.read_csv('../Data/topic_distribution.csv')
full_data_sources = pd.read_csv('../Data/final-news-data.csv')
all_facts = pd.read_csv('../Data/AllFacts.csv')

In [4]:
topic_distribution.head()

,country,country_name,ENVIRONMENT/ENERGY,INTERNATIONAL,POLITICS,EDUCATION/FAMILY,SPORTS,TECHNOLOGY/SCIENCE/SOCIAL MEDIA,SOCIAL_LIFE/DAILY,ENTERTAINMENT/ART/MAGAZINE,COMPANY/BUSINESS,ECONOMY,POLICE/ACCIDENT/VIOLENCE,LEGAL/LAW,HEALTH/MEDICAL
0,US,United States,0.022056,NaN,0.194978,NaN,0.113442,0.082864,0.132510,0.055243,0.084825,0.084809,0.084572,0.056618,0.088082
1,CA,Canada,0.194973,NaN,NaN,NaN,0.095840,NaN,0.146600,0.073039,0.073039,0.138273,0.165221,NaN,0.113016
2,GB,United Kingdom,NaN,0.175466,NaN,NaN,0.228910,NaN,0.057249,0.147708,0.148592,0.037203,NaN,0.121004,0.083868
3,IE,Ireland,NaN,NaN,0.177962,NaN,0.252041,NaN,0.054292,0.184377,NaN,0.143317,NaN,0.133860,0.054152
4,AU,Australia,NaN,0.029435,NaN,NaN,0.124271,0.163525,0.112324,0.153065,NaN,0.157716,NaN,0.117698,0.141965


In [5]:
full_data_sources.head()

,textID,#words,date,country,website,url,title,Topics
0,3732490,815,2015-11-01,US,NPR,http://www.npr.org/2015/11/01/450889721/the-ma...,The Madonna Of 115th Street Gets A Long-Awaite...,SOCIAL_LIFE/DAILY
1,3732492,258,2015-11-01,US,Huffington Post,http://www.huffingtonpost.com/entry/university...,University Of Louisville Sorry A Bunch Of Its ...,SOCIAL_LIFE/DAILY
2,3732496,489,2015-11-01,US,Bleacher Report,http://bleacherreport.com/articles/2584708-kar...,'Kareem: Minority of One' HBO Documentary Prev...,POLITICS
3,3732501,840,2015-11-01,US,VentureBeat,http://venturebeat.com/2015/11/01/what-big-ind...,What big industry will do to the Internet of T...,HEALTH/MEDICAL
4,3732502,470,2015-11-01,US,Tech Insider,http://www.techinsider.io/daenerys-game-of-thr...,Daenerys has been traveling in the wrong direc...,SPORTS


In [6]:
all_facts.head(2)
all_facts.loc[all_facts['Country']=='Tanzania','Unemployment_rate']=None

In [29]:
TOPIC_LIST = ['ENVIRONMENT/ENERGY','INTERNATIONAL','POLITICS','EDUCATION/FAMILY', \
              'SPORTS', 'TECHNOLOGY/SCIENCE/SOCIAL MEDIA', 'SOCIAL_LIFE/DAILY', \
              'ENTERTAINMENT/ART/MAGAZINE', 'COMPANY/BUSINESS', 'ECONOMY', \
              'POLICE/ACCIDENT/VIOLENCE', 'LEGAL/LAW','HEALTH/MEDICAL']

In [8]:
# Correlation 
merged_data = topic_distribution.merge(all_facts, left_on='country_name', right_on='Country')
merged_data.head(2)
data = merged_data

In [9]:
data.columns

Index(['country', 'country_name', 'ENVIRONMENT/ENERGY', 'INTERNATIONAL',
       'POLITICS', 'EDUCATION/FAMILY', 'SPORTS',
       'TECHNOLOGY/SCIENCE/SOCIAL MEDIA', 'SOCIAL_LIFE/DAILY',
       'ENTERTAINMENT/ART/MAGAZINE', 'COMPANY/BUSINESS', 'ECONOMY',
       'POLICE/ACCIDENT/VIOLENCE', 'LEGAL/LAW', 'HEALTH/MEDICAL', 'Country',
       'Population', 'Age_structure0-14', 'Age_structure15-24',
       'Age_structure25-54', 'Age_structure55-64', 'Age_structureover65',
       'Median_age', 'Population_growth_rate', 'Birth_rate', 'Death_rate',
       'Net_migration_rate', 'Sex_ratio', 'Life_expectancy_at_birth',
       'GDP_per_capita', 'Unemployment_rate', 'Inflation_rate',
       'Electricity_renewable_sources', 'Carbon_dioxide_emissions',
       'Internet_users', 'Religions', 'Ethnic_groups'],
      dtype='object')

In [10]:
fact_columns = []
for i in data.columns:
    if(i not in TOPIC_LIST and i not in ['country','country_name','Country']):
        fact_columns.append(i)  
#fact_columns

In [11]:
# Topic Distribution per country
traces=[]
for topic in TOPIC_LIST:
    trace1 = go.Bar(
        x=topic_distribution.country_name,
        y=topic_distribution[topic],
        name=topic
    )
    traces.append(trace1)
layout = go.Layout(
    barmode='stack'
)

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='stacked-bar',colorscale='Light24')

In [13]:
data.describe()

,ENVIRONMENT/ENERGY,INTERNATIONAL,POLITICS,EDUCATION/FAMILY,SPORTS,TECHNOLOGY/SCIENCE/SOCIAL MEDIA,SOCIAL_LIFE/DAILY,ENTERTAINMENT/ART/MAGAZINE,COMPANY/BUSINESS,ECONOMY,...,Death_rate,Net_migration_rate,Sex_ratio,Life_expectancy_at_birth,GDP_per_capita,Unemployment_rate,Inflation_rate,Electricity_renewable_sources,Carbon_dioxide_emissions,Internet_users
count,13.000000,12.000000,14.000000,2.000000,16.000000,12.000000,17.000000,14.000000,12.000000,14.000000,...,20.000000,20.000000,20.000000,20.000000,20.000000,19.000000,20.000000,20.000000,20.000000,20.000000
mean,0.108265,0.116479,0.141781,0.175163,0.146879,0.118442,0.120613,0.132206,0.108121,0.121970,...,7.250000,1.145000,0.992000,73.390000,27365.000000,9.768421,4.185000,6.435000,516.615000,54.615000
std,0.072714,0.068380,0.096226,0.189915,0.054257,0.060361,0.051978,0.054853,0.064703,0.046261,...,1.910773,4.040124,0.043359,8.702141,26282.479155,9.853372,4.754198,6.887385,1226.249218,28.845966
min,0.014033,0.017842,0.033522,0.040873,0.070796,0.028847,0.054292,0.042077,0.017421,0.037203,...,3.500000,-4.500000,0.870000,53.400000,3100.000000,2.100000,-0.800000,0.000000,10.000000,5.400000
25%,0.027872,0.076634,0.082038,0.108018,0.109042,0.076383,0.101459,0.100629,0.057972,0.085685,...,6.350000,-1.325000,0.970000,67.425000,5700.000000,4.800000,1.375000,0.625000,29.500000,29.000000
50%,0.113679,0.114375,0.114340,0.175163,0.138219,0.112088,0.116785,0.134671,0.100773,0.139888,...,7.150000,-0.200000,0.990000,74.300000,12200.000000,5.800000,2.550000,4.900000,93.500000,49.650000
75%,0.157019,0.155383,0.175151,0.242308,0.187873,0.165024,0.133289,0.151726,0.131302,0.157053,...,7.900000,2.850000,1.010000,80.900000,46850.000000,8.200000,5.600000,11.375000,409.250000,82.725000
max,0.234718,0.256281,0.347152,0.309453,0.252041,0.215233,0.287890,0.243091,0.224668,0.176033,...,12.700000,13.600000,1.080000,85.000000,87100.000000,40.000000,17.800000,25.000000,5402.000000,92.000000


In [14]:
data.isnull().any()

country                            False
country_name                       False
ENVIRONMENT/ENERGY                  True
INTERNATIONAL                       True
POLITICS                            True
EDUCATION/FAMILY                    True
SPORTS                              True
TECHNOLOGY/SCIENCE/SOCIAL MEDIA     True
SOCIAL_LIFE/DAILY                   True
ENTERTAINMENT/ART/MAGAZINE          True
COMPANY/BUSINESS                    True
ECONOMY                             True
POLICE/ACCIDENT/VIOLENCE            True
LEGAL/LAW                           True
HEALTH/MEDICAL                      True
Country                            False
Population                         False
Age_structure0-14                  False
Age_structure15-24                 False
Age_structure25-54                 False
Age_structure55-64                 False
Age_structureover65                False
Median_age                         False
Population_growth_rate             False
Birth_rate      

In [15]:
correlations = pd.DataFrame(fact_columns[:-2]) 
for col in TOPIC_LIST:
    correlations[col] = None
correlations.rename(columns={0:'facts'},inplace=True)
#correlations.head()

In [16]:
for topics in TOPIC_LIST:
    for fact in fact_columns:
        if(fact not in ['Religions','Ethnic_groups']):
            dfx = data[[topics,fact]].copy()
            dfx.dropna(inplace=True)
            corr,pvalue = pearsonr(dfx[topics],dfx[fact])
            #if(topics== 'EDUCATION/FAMILY'):
            #    print(fact,topics,corr)
            correlations.loc[correlations['facts']==fact,[topics]] = corr
            #topics_distribution.loc[topics_distribution['country'] == country, topic] = freq
            #print ('Pearson Correlation: corr_val: ',corr,' p_value: ',pvalue)
display(correlations)

,facts,ENVIRONMENT/ENERGY,INTERNATIONAL,POLITICS,EDUCATION/FAMILY,SPORTS,TECHNOLOGY/SCIENCE/SOCIAL MEDIA,SOCIAL_LIFE/DAILY,ENTERTAINMENT/ART/MAGAZINE,COMPANY/BUSINESS,ECONOMY,POLICE/ACCIDENT/VIOLENCE,LEGAL/LAW,HEALTH/MEDICAL
0,Population,-0.195775,-0.353599,0.681059,1,-0.448056,-0.115167,-0.0292559,-0.171461,0.0730816,0.198324,0.0185481,-0.476752,-0.0864549
1,Age_structure0-14,0.101639,-0.163703,-0.0937807,1,-0.0976826,-0.05245,-0.253222,0.159677,3.52974e-05,0.102718,-0.0787227,-0.233801,-0.0261136
2,Age_structure15-24,-0.294921,0.119458,-0.031153,1,-0.294061,-0.0208893,0.307206,0.0963701,-0.170249,0.241123,0.20466,-0.207573,0.0426952
3,Age_structure25-54,-0.20697,0.165333,0.118932,-1,0.12383,0.179448,0.462276,-0.0146373,0.116885,0.0796669,0.115883,0.246602,-0.314655
4,Age_structure55-64,0.119748,0.0626584,0.0730539,-1,0.0896064,-0.0338217,-0.0136919,-0.250382,0.064515,-0.164957,-0.038528,0.238302,0.0506041
5,Age_structureover65,0.106758,0.0452904,0.0307126,1,0.195257,-0.0481056,-0.144655,-0.162931,-0.0398445,-0.255801,-0.0609749,0.185004,0.226112
6,Median_age,0.0852782,0.0949328,0.0822078,-1,0.162537,-0.0261639,-0.0357297,-0.189818,0.0447115,-0.175394,-0.0424183,0.240701,0.0956667
7,Population_growth_rate,-0.0215485,-0.326654,0.00638466,1,-0.0251444,0.476427,0.208092,0.234625,-0.0627234,0.181771,0.0857019,-0.165102,-0.656698
8,Birth_rate,0.0764439,-0.214087,-0.106663,1,-0.0120418,-0.00713576,-0.291278,0.222467,-0.090266,0.129149,-0.00127617,-0.245597,-0.0801741
9,Death_rate,-0.0245412,-0.146422,-0.0644544,1,0.212351,-0.562923,-0.557385,-0.105009,0.0479196,-0.0733967,-0.0110738,-0.502091,0.539506


In [30]:
corr_result = correlations.drop(columns=['facts','EDUCATION/FAMILY'])
new_topics =TOPIC_LIST
new_topics.remove('EDUCATION/FAMILY')
res= corr_result.values
#len(res)
#res

In [32]:
new_topics

['ENVIRONMENT/ENERGY',
 'INTERNATIONAL',
 'POLITICS',
 'SPORTS',
 'TECHNOLOGY/SCIENCE/SOCIAL MEDIA',
 'SOCIAL_LIFE/DAILY',
 'ENTERTAINMENT/ART/MAGAZINE',
 'COMPANY/BUSINESS',
 'ECONOMY',
 'POLICE/ACCIDENT/VIOLENCE',
 'LEGAL/LAW',
 'HEALTH/MEDICAL']

In [31]:
trace = go.Heatmap(z=res,
                   x=new_topics,
                   y=fact_columns[:-2])
data=[trace]
py.iplot(data, filename='labelled-heatmap')

In [ ]:
#correlations = pd.DataFrame(fact_columns+TOPIC_LIST) 
#for col in fact_columns+TOPIC_LIST:
#    correlations[col] = None
#correlations.rename(columns={0:'corr'},inplace=True)
#print(correlations.shape)
#correlations.head()
